In [45]:
import numpy as np
import pandas as pd
import doubleml as dml

In [118]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import FunctionTransformer


# Custom Transformer to subsample labeled observations
class SubsampleTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, subsample_fraction=0.5):
        self.subsample_fraction = subsample_fraction

    def fit(self, X, y=None):
        return self  # Nothing to fit in this transformer

    def transform(self, X, y=None):
        if y is None:
            return X  # Only for fit_transform we need X, y together
        assert all(elements in (1, 0) for elements in y), 'Labels must be binary'

        treated = y == 1
        control = y == 0

        control_subsample = control
        p = [self.subsample_fraction, 1 - self.subsample_fraction]
        random_subsample = np.random.choice([True, False], size=control.sum(), p=p)
        control_subsample[control] = random_subsample

        indices = np.where(treated | control_subsample)[0]
        return X[indices], y[indices]

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y)
        return self.transform(X, y)

# Custom Classifier that rescales predictions
class RescalePredictionsClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, base_classifier=None, rescale_factor=1.0):
        self.base_classifier = base_classifier if base_classifier else LogisticRegression()
        self.rescale_factor = rescale_factor

    def fit(self, X, y):
        self.base_classifier.fit(X, y)
        return self

    def predict(self, X):
        predictions = self.base_classifier.predict_proba(X)
        return np.argmax(predictions, axis=1)

    def predict_proba(self, X):
        base_proba = self.base_classifier.predict_proba(X)
        proba = self.rescale_factor * base_proba / (self.rescale_factor * base_proba + (1 - base_proba))
        # proba = np.column_stack([1 - proba, proba])
        return proba


# Function to apply the transformer only during fit
def fit_transform_only(transformer, X, y=None):
    return transformer.fit_transform(X, y)


In [119]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
import numpy as np

class SubsampleTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, subsample_fraction=0.5):
        self.subsample_fraction = subsample_fraction

    def fit(self, X, y=None):
        return self  # Nothing to fit in this transformer

    def transform(self, X, y=None):
        if y is None:
            return X  # Only for fit_transform we need X, y together
        assert all(elements in (1, 0) for elements in y), 'Labels must be binary'

        treated = y == 1
        control = y == 0

        control_subsample = control
        p = [self.subsample_fraction, 1 - self.subsample_fraction]
        random_subsample = np.random.choice([True, False], size=control.sum(), p=p)
        control_subsample[control] = random_subsample

        indices = np.where(treated | control_subsample)[0]
        return X[indices], y[indices]

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y)
        return self.transform(X, y)

class RescalePredictionsClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, base_classifier=None, rescale_factor=1.0):
        self.base_classifier = base_classifier if base_classifier else LogisticRegression()
        self.rescale_factor = rescale_factor

    def fit(self, X, y):
        self.base_classifier.fit(X, y)
        return self

    def predict(self, X):
        predictions = self.predict_proba(X)
        return np.argmax(predictions, axis=1)

    def predict_proba(self, X):
        base_proba = self.base_classifier.predict_proba(X)[:, 1]
        proba = self.rescale_factor * base_proba / (self.rescale_factor * base_proba + (1 - base_proba))
        all_proba = np.column_stack([1 - proba, proba])
        return all_proba

# Function to apply the transformer only during fit
def fit_transform_only(X, y=None, transformer=None):
    return transformer.fit_transform(X, y)

# Create the pipeline
pipeline = Pipeline([
    ('transformer', FunctionTransformer(func=fit_transform_only, kw_args={'transformer': SubsampleTransformer(subsample_fraction=1.0)}, validate=False)),
    ('classifier', RescalePredictionsClassifier(rescale_factor=1.0))
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Use predict on the full pipeline
y_pred = pipeline.predict_proba(X_test)
y_pred

array([[0.31668643, 0.68331357],
       [0.99773922, 0.00226078],
       [0.98726358, 0.01273642],
       ...,
       [0.98592248, 0.01407752],
       [0.99484974, 0.00515026],
       [0.98243718, 0.01756282]])

In [120]:
# load data from csv
dgp_name = "unbalanced"

df_train = pd.read_csv(f"../dgps/data/{dgp_name}_train.csv")
df_test = pd.read_csv(f"../dgps/data/{dgp_name}_test.csv")

print(f"percentage of treated in train set: {df_train['D'].mean()}")
print(f"percentage of treated in test set: {df_test['D'].mean()}")

df_train.head()

percentage of treated in train set: 0.0534
percentage of treated in test set: 0.06


,X1,X2,X3,D,Y,m_oracle,m_hat,m_calibrated,m_oracle_ate_weights,m_oracle_att_weights,m_hat_ate_weights,m_hat_att_weights,m_calibrated_ate_weights,m_calibrated_att_weights
0,1.372271,0.595549,0,0,3.728639,0.135755,0.139917,0.142857,1.157079,0.157079,1.162679,0.162679,1.166667,0.166667
1,0.462103,0.755682,0,0,1.338655,0.021218,0.023815,0.023364,1.021678,0.021678,1.024396,0.024396,1.023923,0.023923
2,-0.960046,0.240038,1,0,2.427715,0.003470,0.003420,0.001828,1.003482,0.003482,1.003432,0.003432,1.001832,0.001832
3,-0.607569,0.934605,1,0,2.387937,0.003506,0.004413,0.001828,1.003518,0.003518,1.004433,0.004433,1.001832,0.001832
4,-0.565393,0.395288,0,0,1.465630,0.003966,0.004019,0.001828,1.003982,0.003982,1.004035,0.004035,1.001832,0.001832


In [121]:

fraction = df_train['D'].mean()

# Create the pipeline
pipeline = Pipeline([
    ('transformer', FunctionTransformer(func=fit_transform_only, kw_args={'transformer': SubsampleTransformer(subsample_fraction=fraction)}, validate=False)),
    ('classifier', RescalePredictionsClassifier(rescale_factor=fraction)),
])


X_train = df_train[['X1', 'X2', 'X3']].values
y_train = df_train['D'].values

X_test = df_test[['X1', 'X2', 'X3']].values

In [122]:
# Fit the pipeline
pipeline.fit(X_train, y_train)

# Use predict on the full pipeline
y_pred = pipeline.predict_proba(X_test)

In [123]:
y_pred[0:10, 1]

array([0.10331679, 0.00012098, 0.00068842, 0.00072548, 0.01032256,
       0.00012469, 0.00084842, 0.00110119, 0.0022832 , 0.00071451])

In [124]:
df_test.head(10)

,X1,X2,X3,D,Y,m_oracle,m_hat,m_calibrated,m_oracle_ate_weights,m_oracle_att_weights,m_hat_ate_weights,m_hat_att_weights,m_calibrated_ate_weights,m_calibrated_att_weights
0,2.637882,0.457121,0,1,7.535259,0.693948,0.683314,0.769231,1.441030,1.000000,1.463457,1.000000,1.300000,1.000000
1,-1.037066,0.655834,1,0,0.056473,0.001966,0.002261,0.003219,1.001970,0.001970,1.002266,0.002266,1.003229,0.003229
2,-0.349317,0.052052,1,0,2.743051,0.014054,0.012736,0.009804,1.014254,0.014254,1.012901,0.012901,1.009901,0.009901
3,0.001619,0.243824,0,0,2.590433,0.014195,0.013413,0.009804,1.014399,0.014399,1.013596,0.013596,1.009901,0.009901
4,1.074333,0.197206,1,0,6.203856,0.175297,0.163406,0.198473,1.212558,0.212558,1.195323,0.195323,1.247619,0.247619
5,-0.925890,0.135362,0,0,1.463885,0.002505,0.002330,0.003219,1.002511,0.002511,1.002335,0.002335,1.003229,0.003229
6,0.235615,0.725759,0,0,1.355423,0.014001,0.015653,0.020833,1.014200,0.014200,1.015902,0.015902,1.021277,0.021277
7,0.149456,0.044816,0,0,3.330547,0.023069,0.020227,0.020833,1.023614,0.023614,1.020644,0.020644,1.021277,0.021277
8,0.683212,0.554894,0,0,3.627412,0.039602,0.041093,0.048433,1.041235,0.041235,1.042854,0.042854,1.050898,0.050898
9,-0.055548,0.909041,1,0,2.437770,0.010770,0.013213,0.009804,1.010888,0.010888,1.013390,0.013390,1.009901,0.009901


In [125]:
dml_data = dml.DoubleMLData(df_train, 'Y', 'D', ['X1', 'X2', 'X3'])

In [128]:
dml_model = dml.DoubleMLIRM(dml_data,
                            ml_g=LinearRegression(),
                            ml_m=LogisticRegression(),
                            n_folds=5)

dml_model.fit()
print(dml_model)

================== DoubleMLIRM Object ==================

------------------ Data summary      ------------------
Outcome variable: Y
Treatment variable(s): ['D']
Covariates: ['X1', 'X2', 'X3']
Instrument variable(s): None
No. Observations: 5000

------------------ Score & algorithm ------------------
Score function: ATE

------------------ Machine learner   ------------------
Learner ml_g: LinearRegression()
Learner ml_m: LogisticRegression()
Out-of-sample Performance:
Regression:
Learner ml_g0 RMSE: [[0.50367412]]
Learner ml_g1 RMSE: [[0.50319678]]
Classification:
Learner ml_m Log Loss: [[0.14389963]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err         t  P>|t|     2.5 %    97.5 %
D  1.972574  0.040441  48.77695    0.0  1.893311  2.051836


In [130]:
dml_model = dml.DoubleMLIRM(dml_data,
                            ml_g=LinearRegression(),
                            ml_m=Pipeline([
                                ('transformer', FunctionTransformer(func=fit_transform_only, kw_args={'transformer': SubsampleTransformer(subsample_fraction=fraction)}, validate=False)),
                                ('classifier', RescalePredictionsClassifier(rescale_factor=fraction)),
                            ]),
                            n_folds=5)

dml_model.fit()
print(dml_model)

AttributeError: 'RescalePredictionsClassifier' object has no attribute 'classes_'